# Testing the trained U-Net models

## Module imports

In [16]:
#import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm # progress bar
from torch.optim import Adam
import numpy as np
from skimage import metrics

# custom modules
from unet import StentDataset, UNet, ImageAugmentation

## Loading the Model

In [9]:
!ls saved_models/unet-exp1-iter_24000.pt

saved_models/unet-exp1-iter_24000.pt


In [13]:
model_save_path = "/home/igor/Work/CS/PP22/Medical-Noise-Cancellation/saved_models/unet-exp1-iter_24000.pt"
model = UNet(in_channels=1, out_channels=1)
model.load_state_dict(torch.load(model_save_path, map_location=torch.device("cpu")))
model.eval()  # Switching to evaluation mode

UNet(
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_down_0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
  )
  (conv_down_4): Sequential(
    (0): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(

## Generating test data

In [19]:
n_test_images = 10

# custom image generator
image_generator = ImageAugmentation(base_image_path="data/dataset/base_png")

# generate test images
image_generator.generate_input_images(n_images=n_test_images,
                                      save_path="data/dataset/test",
                                      generate_input=True,
                                      print_progress=False)

Generating 10 output images...
Print progress disabled since less than 100 images are generated.
10 output images generated at data/dataset/test/y
Generating 10 input images with noise...
10 input images generated at data/dataset/test/x


In [18]:
batch_size = 2
test_dataset = StentDataset(input_path="data/dataset/test/x",
                            target_path="data/dataset/test/y")
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/dataset/test/y'

## Testing model

In [1]:
PSNR = 0
SSIM = 0
for i, data in enumerate(tqdm(test_loader)):
    # Get inputs and targets
    inputs, targets = data
    # Calculate outputs
    outputs = model(inputs)
    # Preprocess output tensors and targets for scikit-image
    outputs = outputs.detach().numpy()[:, 0, delta:-delta, delta:-delta]
    targets = targets.detach().numpy()[:, 0, delta:-delta, delta:-delta]
    # Calculate metrics
    for j in range(batch_size):
        output = outputs[j, ...]
        target = targets[j, ...]
        PSNR = PSNR + metrics.peak_signal_noise_ratio(target, output)/n_test_images
        SSIM = SSIM + metrics.structural_similarity(target, output)/n_test_images

NameError: name 'tqdm' is not defined